## Load alphase from directory into dictionary sturct to pickle

In [10]:
import numpy as np
import matplotlib.pyplot as plt
import pathlib
import pickle

In [11]:
def read_alpha(filename):
    with open(filename,'r') as myfile:
        contents= myfile.read()
    alpha_list = contents.split()
    alpha_list = np.array(alpha_list).astype(float)
    n , nx, ny, nz = alpha_list[0:4].astype(int)
    alpha_list= np.delete(alpha_list, [0,1,2,3])
    a=np.zeros([nx,ny,nz])
    point = -1
    for iz in range(0,nz):
        for iy in range(0,ny):
            for ix in range(0,nx):
                point += 1
                a[ix,iy,iz]=alpha_list[point]
    
    if point+1==n:
        return alpha_list , a
    else:
        print ("Error: lengths dont match")
        print (point, n)
        return 

In [12]:
def load_from_dir(base_path,exclude_list=[".","_",","],pass_suffix='.out',pass_format_flag=False,pass_delim='_',pass_split_num=2):
    #create dictionary
    pbc={}
    #itterate over folders in base_path
    for folder in base_path.iterdir():
        #ignore files
        if folder.is_dir():
            #ignore hidden and special folders
            if folder.name[0] not in exclude_list:
                temp_dir={}
                temp_dir=load_out_files(folder,suffix=pass_suffix,format_flag=pass_format_flag,delim=pass_delim,split_num=pass_split_num)
                parts = folder.stem.split('(') #pass_delim
                #del parts[0:3]
                key=parts[0]
                #key= folder.stem
#                 if len(parts)>1:
#                     for item in parts[1:]:
#                         key += "_"
#                         key += item
                pbc[key]=temp_dir
    return pbc 


In [13]:

def load_out_files(path,suffix='.out',format_flag=False,delim='_',split_num=2):
    #takes path and return dictionary with keys as filenames and 
    path_object=pathlib.Path(path)
    obj_dict={}
    for file in path_object.iterdir():
        if file.name[0] not in [".",",","_"]:
            if file.suffix==suffix:
                key=file.stem
                if format_flag==True:
                    parts=key.split(delim,split_num)
                    key=parts[-1]
                #parts=file.stem.split("_")
                #key = parts[-1]
                
                obj_dict[key]=read_alpha(file)
        
    return obj_dict

In [14]:
def write_alpha_file(alpha_list,filename,seperator='    '):
    nx = round(len(alpha_list)**(1.0/3.0))
    ny = nx 
    nz = nx

    with open(filename,'w') as myfile:
        myfile.writelines('a'+seperator+'b'+seperator+'c'+seperator+'charge\n')
        point = -1
        for iz in range(1,nz+1):
            for iy in range(1,ny+1):
                for ix in range(1,nx+1):
                    point += 1
                    line = str(ix)+seperator+str(iy)+seperator+str(iz)+\
                        seperator+str(alpha_list[point])+' '+ '\n'
                    myfile.writelines(line)
                

In [15]:
def folder_alpha_write(data,seed,path,seperator='    '):
    for alpha in data[seed].keys():
        path_name = pathlib.Path.joinpath(path,str(alpha)+'.charg_frm')
        write_alpha_file(data[seed][alpha][0],path_name,seperator)

In [16]:
base_path = pathlib.Path.cwd()

In [17]:
y = load_from_dir(base_path)

In [18]:
# pickling data into object file
with open('data.obj','wb') as myfile:
     pickle.dump(y,myfile)

In [19]:
# example of how to load data 
with open('data.obj','rb') as myfile:
     z= pickle.load(myfile)

In [20]:
y.keys()

dict_keys(['Ni', 'Cu', 'Co', 'Fe', 'Fe_no_init_spin'])

In [25]:
# writing alpha values

In [26]:
temp_path = base_path.joinpath('Fe')
folder_alpha_write(y,'Fe',temp_path)

In [27]:
temp_path = base_path.joinpath('Ni')
folder_alpha_write(y,'Ni',temp_path)

In [28]:
temp_path = base_path.joinpath('Co')
folder_alpha_write(y,'Co',temp_path)

In [29]:
temp_path = base_path.joinpath('Cu')
folder_alpha_write(y,'Cu',temp_path)

In [30]:
temp_path = base_path.joinpath('Fe_no_init_spin')
folder_alpha_write(y,'Fe_no_init_spin',temp_path) 